In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
from SALib.sample.morris.strategy import Strategy
from SALib.sample.morris import generate_trajectory, _sample_oat

In [ ]:
sample = _sample_oat({'num_vars': 10}, 5)

In [ ]:
sample.shape

In [ ]:
sample[:11, 4]

## `sample` matrix

The way that this array is constructed is... unintuituve. Maybe it is like this for backwards compatibility. `sample` is a two-dimensional array which includes three dimensions: trajectory, parameter, and parameter value. Personally, I would prefer this to be a three dimensional array (order doesn't really matter).

Note: My (older, unmaintained) [py-elem-effects](https://bitbucket.org/cmutel/py-elem-effects/) library doesn't store the coordinates of every single point at each point on the trajectory, but rather the starting and ending values for each parameter, and the order the parameters change in. This means you reduce memory usage substantially.

The rows are parameter values for the first trajectory, follow by for the second trajectory, etc. Columns are the parameters.

## Trajectory generation performance

The function `_sample_oat` includes this line:

    sample = np.array([generate_trajectory(group_membership, num_levels)
                       for n in range(N)])

These types of loops should almost always be avoided, as there are numpy operators that are much faster. Let's see if we can make trajectory generation faster.

In [ ]:
m = np.asmatrix(np.identity(1000, dtype=int))

In [ ]:
%timeit generate_trajectory(m, 4)

In [ ]:
type(generate_trajectory(m, 4))

In [ ]:
%timeit generate_trajectory_2(1000, 4)

In [9]:
quickref


In [24]:
_dh

['/Users/akim/Documents/projects/sa', '/Users/akim/Documents/projects']

In [8]:
alias

Total number of aliases: 1449


[('2to3', '2to3'),
 ('2to3-', '2to3-'),
 ('2to3-27', '2to3-2.7'),
 ('2to3-37', '2to3-3.7'),
 ('7z', '7z'),
 ('7za', '7za'),
 ('7zr', '7zr'),
 ('AppleFileServer', 'AppleFileServer'),
 ('AssetCacheLocatorUtil', 'AssetCacheLocatorUtil'),
 ('AssetCacheManagerUtil', 'AssetCacheManagerUtil'),
 ('AssetCacheTetheratorUtil', 'AssetCacheTetheratorUtil'),
 ('BootCacheControl', 'BootCacheControl'),
 ('BuildStrings', 'BuildStrings'),
 ('CpMac', 'CpMac'),
 ('DeRez', 'DeRez'),
 ('DevToolsSecurity', 'DevToolsSecurity'),
 ('DirectoryService', 'DirectoryService'),
 ('FileStatsAgent', 'FileStatsAgent'),
 ('GetFileInfo', 'GetFileInfo'),
 ('IOAccelMemory', 'IOAccelMemory'),
 ('KernelEventAgent', 'KernelEventAgent'),
 ('MergePef', 'MergePef'),
 ('MvMac', 'MvMac'),
 ('NetBootClientStatus', 'NetBootClientStatus'),
 ('PasswordService', 'PasswordService'),
 ('ResMerger', 'ResMerger'),
 ('Rez', 'Rez'),
 ('RezDet', 'RezDet'),
 ('RezWack', 'RezWack'),
 ('SafeEjectGPU', 'SafeEjectGPU'),
 ('SetFile', 'SetFile'),
 ('